# Why LSTM outperforms RNN when dealing with long temporal sequences?

We have already see the power of Artificial Neural Networks. Feedforward ANNs can be used to perform fucntion fitting, digits recognition, image recognition... But there are tasks that Feedforward ANNs can't deal with well. Semantic analysis is one of the scenarios that Feedforward would fail. In semantic analysis, the preceding information and the order of information have important influence on semantic judgment. Feedforward can't learn that kind of sequential rules. In fact, any tast in which the temporal sequence matters, Feedforward can't perform well. 

That's why we need Recurrent Neural Networks(RNNs) and Long-short Term Memorys(LSTMs) to solve this kind of problems. The major difference between Feedforward Neural Networks and Recurrent Neural Networks(or Long-short Term Memorys) is RNNs(or LSTMs) have **hidden state**. The **hidden state** of a neuron at one moment passes on to the next and serves as an input to the neuron at the next moment.

Let's take a look at RNN first.

## Recurrent Neural Networks

The mechanism of RNN is quite simple. RNNs take the input **$x_t$** of the current moment **$t$** as one of the inputs and take the hidden state **$h_{t-1}$** as another input. Then ANNs use those two inputs calculating the hidden state of now ( **$h_t$** ) and the output **y_t**. Let's look at it methamatically:

Calculating the hidden state:
$$ h_t = f(W_I x_t + W_h h_{t-1})$$

Here **$f(\dot)$** is the activation function, **$W_I$** is the weight matrix for imput **$x$** , and **$W_h$** is the weight matrix for previous hidden state **$h_{t-1}$**. 

Calculating the output state:
$$ O = W_O h_t$$

Here **$W_O$** is the weight matrix for **$h_t$**.

RNN looks pretty simple and elegant, but the fact is, RNN can't learn long-term sequential rules or temporal rules because RNN would encouter the notorious gradient explode or gradient vanish problem. Luckly, there is another option here --- LSTM which I will introduce it later. LSTM won't encounter that severe gradient vanish or explode problem ao it can perform long-term prediction and semantic analysis of long sentences or paragraphs.

Before I introduce why the LSTM is good, I would like to tell you why RNNs fail first.

## Why RNNs get the gradient vanish / explode problem?

If we use RNN to deal with a T length time sequence, we need to calculate the derivative of cost funciton **$J$** with respect to weight matrix **$W_I$**, **$W_h$** and **$W_O$** . Let's take the weight matrix for hidden state for example to see why it would have that gradient explode or gradient vanish issue.

$$\frac{dJ}{dW_h} = \sum_{t=0}^T \frac{dJ}{dO_T} \frac{dO_T}{dh_T} \frac{\partial{h_T}}{\partial{h_t}} \frac{\partial{h_t}}{\partial{W_h}}$$

Where, 

$$\frac{h_T}{h_t} = \frac{h_T}{h_{T-1}} \frac{\partial{h_{T-1}}}{\partial{h_{T-2}}} ... \frac{\partial{h_{t+1}}}{\partial{h_t}}$$

Now we still can't see why we would have gradient vanish or gradient explode yet. Let's take a close look at $\frac{\partial{h_{t+1}}}{h_t}$.

$$\frac{\partial{h_{t+1}}}{\partial{h_{t}}} = diag(f'(W_I x_{t+1} + W_h h_t))W_h$$

Where **$diag(v)$** denotes the diagonal matrix in which **$v$** is the main diagonal and **$f'(\bullet)$** denotes the derivative of the activation function.

Since the weight matrix **$W_h$** for all time slot t are the same, the partial derivative $\frac{\partial{h_T}}{\partial{h_t}}$ is basically the repeated matrix multiplication of **$W_h$** and those derivative $f'(\bullet)$ are actually scaling factors to the matrix **$W_h$**.

Now we can see that if the dominant eigenvalue of matrix **$W_h$** is bigger than 1, the backpropagation of **$W_h$** is likely to have gradient explode issue; if the dominant eigenvalue of matrix **$W_h$** is less than 1, then the backpropagation is likely to have gradient vanish issue.

If that is not quite clear to you, than you can look at the following introduction.

## Eigenvalue and repeated matrix multiplication

Now let me show you how the eigenvalue of a matri would affects its repeated multiplication. **$F$** is a $m \times m$ diagonalizable matrix. And I want to calculate the repeated multiplication of it.

$$F^2 = F F$$

We can perform eigendecomposition to matrix **$F$**.

$$F = Q \Lambda Q^{-1}$$

Where **$\Lambda$** is the eigenvalue matrix. The elements in the main diagonal of matrix **$\Lambda$** is the eigenvalues of matrix **$F$** . And matrix **$Q$** is eigen vector matrix. The row vectors in matrix **$Q$** are eigenvectors of matrix **$F$** . So we have,

$$F^2 = Q \Lambda Q^{-1} Q \Lambda Q^{-1} = Q \Lambda^2 Q^{-1}$$

And for **n** time repeated multiplication, we have

$$F^n = Q \Lambda^n Q^{-1}$$

Take the following repeated matrix multiplication for example:

In [1]:
import numpy as np

In [2]:
F = np.array([[0,1],[1,1]])
F

array([[0, 1],
       [1, 1]])

In [4]:
v, Q = np.linalg.eig(F)
print('Eigenvalues of matrix F is:')
print(v)
print()
print('Eigenvectors of matrix F is:')
print(Q)

Eigenvalues of matrix F is:
[-0.61803399  1.61803399]

Eigenvectors of matrix F is:
[[-0.85065081 -0.52573111]
 [ 0.52573111 -0.85065081]]


In [8]:
n = 20
Lambda = np.diag(v)
Lambda

array([[-0.61803399,  0.        ],
       [ 0.        ,  1.61803399]])

We can see, the demo matrix **$F$** has two eigenvalues, one is above 1, and the other is less than 1(absolute value). Let's see how the repeated multiplication of matrix **$\Lambda$** woul be.

In [9]:
Lambda_n = Lambda**n
Lambda_n

array([[6.61069614e-05, 0.00000000e+00],
       [0.00000000e+00, 1.51269999e+04]])

We can already see the explode here.

In [10]:
F_n = np.matmul(np.matmul(Q, Lambda_n), np.linalg.inv(Q))
F_n

array([[ 4181.,  6765.],
       [ 6765., 10946.]])

As you can see, the dominant eigenvalue of matrix **$F$** is bigger than 1, so the repeated multiplication ended up exploded.

So if you look back, and you will see why the dominant eigenvalue of matrix **$W_h$** would suffer gradient vanish or explode when backpropagation.

You may ask, well the repeated multiplication of matrix **$W_h$** do seems to be vanish or explode while doing repeated multiplication, but there is a derivative **$f'(\bullet)$** we neglected.

Well, as I mentioned before, that derivative **$f(\bullet)$** acts like a scaling factor to the matrix and it do may affect the propagation. Now let's see how the derivative affect the backpropagation.

If the activation function **$f(\bullet)$** is the identity activation function, then the $\frac{\partial{h_{T}}}{\partial{h_{t}}}$ is exactly the repeated multiplication of matrix **$W_h$** and whether or not it would suffer gradient vanish or gradient explode depends entirelly on the dominant eigenvalue of matrix **$W_h$**.

$$\frac{\partial{h_{T}}}{\partial{h_{t}}} = W_h^{T-t}$$

If the activation **$f(\bullet)$** is **sigmoid** or **tanh**, the derivative of them are both less than 1, so if we multiply it to matrix **$W_h$** , it definatly would make the gradient vanish more quickly if the dominant eigenvalue of matrix **$W_h$** is less than 1. If the dominant eigenvalue of matrix **$W_h$** is bigger than 1, then we wouldn't be so sure whether the gradient would explode or vanish. But one thing is quite certian, if the secquence go longer, the gradient vanish or gradient explode issue is inevitable.

**Now we already know why RNN would suffer from gradient vanish or gradient explode. Let's take a look at why LSTM is good at dealing with long temporal sequences.**

## Long-short Term Memory(LSTM)
LSTM is actually also a special kind of neural network. The major difference between LSTM and RNN is that LSTM has a special mechanism called **forget gate**. As RNNs do, LSTMs also have **hidden state** that would pass to the next time slot. But not like RNN, iformations are selected through the forget gate and only let through those informations that is usefull. Let see how this would help LSTM get away from gradient vanish or gradient explode.

### Input gate in LSTM
First, the input is squashed between -1 and 1 using a tanh activation function. This can be expressed by:

$$g = tanh(b^g+x_tU^g+y_{t-1}V^g)$$

Where **$U^g$** and **$V^g$** are the weights for the input and previous cell output, respectively, and **$b^g$** is the input bias. Note that the exponents **g** are not a raised power, but rather signify that these are the input weights and bias values (as opposed to the input gate, forget gate, output gate etc.).

This squashed input is then multiplied element-wise by the output of the input gate. The input gate is basically a hidden layer of sigmoid activated nodes, with weighted **$x_t$** and **$y_{t-1}$** input values, which outputs values of between 0 and 1 and when multiplied element-wise by the input determines which inputs are switched on and off. In other words, it is a kind of input filter or gate. The expression for the input gate is:

$$i = \sigma(b^i + x_tU^i+ y_{t-1}V^i)$$

### The hidden state and the forget gate
Forget gate is again a sigmoid activated set of nodes which is element-wise multiplied by the hidden state of the previous moment **$s_{t-1}$** to determine which previous states should be remembered (i.e. forget gate output close to 1) and which should be forgotten (i.e. forget gate output close to 0). This allows the LSTM cell to learn appropriate context. The forget gate is like:

$$f = \sigma(b^f + x_tU^f + y_{t-1}V^i)$$

So the hidden state of the current moment is:

$$s_t = s_{t-1}\circ f + g \circ i$$

Where $\circ$ denotes element-wise multiplication.

### The output gate in LSTM

The final stage of the LSTM cell is the output gate. The output gate has two components – another tanh squashing function and an output sigmoid gating function. The output sigmoid gating function, like the other gating functions in the cell, is multiplied by the squashed state st to determine which values of the state are output from the cell. 

The output gate is like:

$$o = \sigma(b^o + x_tU^o + y_{t-1}V^o)$$

So the final output of the cell is:

$$y_t = tanh(s_t)\circ o$$

## Why LSTM won't encounter gradient vanish or gradient explode in BP process?
To understand why, we need to see how the gradient propagated through the network. As we stated before, the recursive derivative is the main thing that is causing the vanishing gradient, so lets expand out the full derivative for the hidden state $\frac{\partial{s_T}}{\partial{s_t}}$. But first, we need to know what is $\frac{\partial{s_{t+1}}}{\partial{t}}$ looks like.

Let's take a look at how we calculate the hidden state again first:

$$s_t = s_{t-1}\circ f_t + g_t \circ i_t$$

You should be noticed that **f, g,** and **i** are all functions of the previous state. So there would be four part in the partial derivative of **$s_{t+1}$** with respect to **$s_t$**:

$$\frac{\partial{s_{t+1}}}{\partial{s_t}} = \frac{\partial{s_{t+1}}}{\partial{s_t}} + \frac{\partial{s_{t+1}}}{\partial{f_{t+1}}} \frac{\partial{f_{t+1}}}{\partial{y_t}}\frac{\partial{y_t}}{\partial{s_t}} + \frac{\partial{s_{t+1}}}{\partial{g_{t+1}}} \frac{\partial{g}}{\partial{y_t}} \frac{\partial{y_t}}{\partial{s_t}} + \frac{\partial{s_{t+1}}}{\partial{i_{t+1}}} \frac{\partial{i}}{\partial{y_t}} \frac{\partial{y_t}}{\partial{s_t}}$$

Now lets explicitly write out these derivatives:

$$frac{\partial{s_{t+1}}}{\partial{s_t}} = f_{t+1} + s_{t}\sigma'(\bullet)V^itanh'(s_t)o_t + i_{t+1}tanh'(\bullet)V^gtanh'(s_t)o_t + g_{t+1}\sigma'(\bullet)V^itanh'(s_t)o_t$$

Now if we want to backpropagate back k time steps, we simply multiply terms in the form of that one above k times. 

Note the big difference between this recursive gradient and the one for RNNs. In RNNs, the terms $\frac{\partial{h_{t+1}}}{\partial{h_t}}$ will eventually take on a values that are either always greater than 1 or less than 1, this is essentially what leads to the vanishing/exploding gradient problem.

While the derivative here $\frac{\partial{s_{t+1}}}{\partial{s_t}}$ at any time step can take on either values that are greater than 1 or less than 1. 

So here is the thing, if you have a sequence of number which are all greater than 1 or all less than 1, and you multiply them one by one, the results would definitely be a huge number or a tiny tiny number if you multiply them enough times. But if you mutiply a sequence of numbers of which some are greater than 1 and some are less than 1(which can be adjusted by the forget gate in this case), you won't be expect to see a clear vanish or explode of the result.

And that's basically how LSTM wouldn't easily encounter gradient descent or gradient explode problem.